In [1]:
import pandas as pd
import numpy as np
from processer import preprocessSmilesList, fragmentSmilesList

# Zinc data

In [2]:
# this .csv is just one of the elaborated dataset
rawData=np.loadtxt("data/zinc.csv", skiprows=1, dtype=str,comments=None)

In [3]:
rawData

array(['CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1',
       'C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1',
       'N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)cc2)cc1', ...,
       'Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1',
       'Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1',
       'O=C(CC(c1ccccc1)c1ccccc1)N1CCN(S(=O)(=O)c2ccccc2[N+](=O)[O-])CC1'],
      dtype='<U120')

In [4]:
# preprocess data removing salts, isotopes, geometric stereochemical and duplicates
prepData=preprocessSmilesList(rawData)

 28%|██████████████████████████████████████████████████████████████▉                                                                                                                                                                 | 70155/249456 [00:00<00:00, 652152.92it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 249456/249456 [00:00<00:00, 447136.61it/s]


In [9]:
prepData.size

249414

In [5]:
# tokenize cured data with default cleavage pattern : exocyclic single bonds
zincT = fragmentSmilesList(prepData)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 249414/249414 [00:04<00:00, 53091.23it/s]


In [11]:
zincT.dropna(inplace=True)

In [12]:
prepData.size-zincT.shape[0]

86

In [10]:
zincT

,smiles,tokens,nToks
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1,"(F, <5>, c1ccccc1, <0>, N, C=O, C, <6>, c1ccc2...",18
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1,"(C, <0S>, C1CCCCC1, <2>, <4S>, (, C, ), N, <1>...",16
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,"(C1CCNC1, <3>, C=O, C|R, (, <0>, c1ccccc1, ), ...",16
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,"(C, C, O, C=O, <1R>, C1CCNCC1, <3>, C=O, <0>, ...",15
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C(O)[C@H](C#N)C...,"(O, <4>, C1=CC2(CC=N1)CCCCC2, <0>, <1>, (, C#N...",20
...,...,...,...
249451,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc21,"(C, <6>, c1ccc2c(c1)CCCC2, <9S>, <6>, (, C, ),...",14
249452,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,"(C, C, (, C, ), (, C, ), C=O, <3>, C1CCNCC1, <...",22
249453,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1,"(c1ccccc1, <0>, C, N, (, C, ), C=O, C=O, N, <3...",18
249454,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1,"(C, <2>, c1cc[nH]c1, <0>, <4>, (, C, ), <3>, (...",22


In [16]:
from functools import partial

In [17]:
from chemicalgof import Smiles2GoF
smartsRotatable="[!$(*#*)&!D1]-!@[!$(*#*)&!D1]"
decomposeSmilesByRotatable = partial(Smiles2GoF, pattBonds = smartsRotatable)

In [18]:
# fncDecompose argument is just in case of non default cleavage pattern
zincRotatable=fragmentSmilesList(prepData, fncDecompose=decomposeSmilesByRotatable)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 249414/249414 [00:04<00:00, 62006.97it/s]


In [19]:
zincRotatable.dropna(inplace=True)

In [21]:
prepData.size-zincRotatable.shape[0]

864

In [20]:
zincRotatable

,smiles,tokens,nToks
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1,"(CC(C)C, <1>, <0>, c1ccc2occc2c1, <6>, C, C=O,...",10.0
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1,"(Cn1cnnc1, <2>, <5>, c1ccncc1, <1>, N, <3>, CC...",8.0
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,"(C1CCNC1, <3>, C=O, C|R, (, <0>, c1ccccc1, ), ...",14.0
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,"(CC, <0>, O, C=O, <1R>, C1CCNCC1, <3>, C=O, <0...",13.0
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C(O)[C@H](C#N)C...,"(Clc1ccccc1, <3>, N, C=O, C, S, <3>, N#CC1=CN=...",8.0
...,...,...,...
249451,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc21,"(CC1(C)CCCc2ccccc21, <5S>, C, N, C=O, C, <2>, ...",8.0
249452,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,"(CC(C)C, <1>, C=O, <3>, C1CCNCC1, <0>, C=O, N,...",14.0
249453,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1,"(c1ccccc1, <0>, C, <1>, CN, <1>, C=O, C=O, N, ...",11.0
249454,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1,"(NC=O, <1>, C, O, <3>, c1ccccc1, <0>, N, C=O, ...",14.0
